### Cleaning and Validating Dirty Sales Data

In [1]:
# Reading the data with multi-index
import pandas as pd
dirty_sales = pd.read_excel(r'Badly-Structured-Sales-Data.xlsx', sheet_name="Dirty 1", header=[0,1])
dirty_sales.head()

Segment>>    Consumer                                       \
      Ship Mode>> First Class Same Day Second Class Standard Class   
0        Order ID         NaN      NaN          NaN            NaN   
1  CA-2011-100293         NaN      NaN          NaN            NaN   
2  CA-2011-100706         NaN      NaN       129.44            NaN   
3  CA-2011-100895         NaN      NaN          NaN         605.47   
4  CA-2011-100916         NaN      NaN          NaN            NaN   

      Consumer Total   Corporate                                       \
  Unnamed: 5_level_1 First Class Same Day Second Class Standard Class   
0                NaN         NaN      NaN          NaN            NaN   
1                NaN         NaN      NaN          NaN            NaN   
2             129.44         NaN      NaN          NaN            NaN   
3             605.47         NaN      NaN          NaN            NaN   
4                NaN         NaN      NaN          NaN         788.86   

      Corporate Total Home Office                                       \
  Unnamed: 10_level_1 First Class Same Day Second Class Standard Class   
0                 NaN         NaN      NaN          NaN            NaN   
1                 NaN         NaN      NaN          NaN         91.056   
2                 NaN         NaN      NaN          NaN            NaN   
3                 NaN         NaN      NaN          NaN            NaN   
4              788.86         NaN      NaN          NaN            NaN   

    Home Office Total  
  Unnamed: 15_level_1  
0                 NaN  
1              91.056  
2                 NaN  
3                 NaN  
4                 NaN

In [2]:
# Drop the first column that has no data and the total columns
dirty_sales.drop(index=dirty_sales.index[0], axis=0, inplace=True) 

In [3]:
# Drop the Totals columns since they only tally one value 
dirty_sales = dirty_sales.drop(columns=['Consumer Total', 'Corporate Total', 'Home Office Total'], level=0)

In [4]:
# Flatten the multilevel columns and convert to str
dirty_sales.columns = dirty_sales.columns.to_flat_index()
dirty_sales.columns = dirty_sales.columns.map('_'.join)
dirty_sales.columns

Index(['Segment>>_Ship Mode>>', 'Consumer_First Class', 'Consumer_Same Day',
       'Consumer_Second Class', 'Consumer_Standard Class',
       'Corporate_First Class', 'Corporate_Same Day', 'Corporate_Second Class',
       'Corporate_Standard Class', 'Home Office_First Class',
       'Home Office_Same Day', 'Home Office_Second Class',
       'Home Office_Standard Class'],
      dtype='object')

In [5]:
# rename first column which will act as our index
dirty_sales = dirty_sales.rename(columns={'Segment>>_Ship Mode>>': 'Order_ID'})

In [6]:
# Reshape the dataframe and drop rows with no sales data
sales = pd.melt(dirty_sales, id_vars='Order_ID', var_name='Ship Mode', value_name='Sale')
sales.dropna(inplace=True)

In [7]:
# remove rows with totaled data 
grand_totals = sales[sales['Order_ID'] == 'Grand Total'].index
sales.drop(grand_totals, inplace=True)
sales.reset_index(drop=True, inplace=True)

In [8]:
# split the ship mode column to categorize our order into different segments and reorder df 
sales[['Segment','Ship Mode']] = sales['Ship Mode'].str.split("_",expand=True)
sales = sales.reindex(columns=['Order_ID', 'Segment', 'Ship Mode', 'Sale'])

In [9]:
sales

,Order_ID,Segment,Ship Mode,Sale
0,CA-2011-103366,Consumer,First Class,149.950
1,CA-2011-109043,Consumer,First Class,243.600
2,CA-2011-113166,Consumer,First Class,9.568
3,CA-2011-124023,Consumer,First Class,8.960
4,CA-2011-130155,Consumer,First Class,34.200
...,...,...,...,...
817,US-2014-129224,Home Office,Standard Class,4.608
818,US-2014-132031,Home Office,Standard Class,513.496
819,US-2014-132297,Home Office,Standard Class,598.310
820,US-2014-132675,Home Office,Standard Class,148.160
